# 03 – Pipeline TDA sobre Series de Criptomonedas
Proyecto: *Análisis topológico de criptomonedas mediante paisajes de persistencia (SEIO 2023)*

Este notebook ejecuta el pipeline completo de Topological Data Analysis (TDA):

1. Carga de datos procesados (retornos logarítmicos).
2. Construcción de nubes de puntos en \( \mathbb{R}^4 \).
3. Cálculo de diagramas de persistencia usando Ripser.
4. Conversión a paisajes de persistencia (PersLandscapeExact).
5. Cálculo de normas L1 y L2 del paisaje asociado a H1.
6. Exportación del resultado como archivo CSV en `data/processed/`.

Compatible con Google Colab.

In [ ]:
# ===============================================
# 1. Configuración inicial
# ===============================================

import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

from ripser import ripser
from persim import PersLandscapeExact

import os

plt.style.use('seaborn-v0_8')

print("Librerías cargadas correctamente.")

## 2. Cargar retornos logarítmicos procesados

Se asume que `02_preprocesamiento.ipynb` generó un archivo con retornos logarítmicos
llamado `retornos_logaritmicos.parquet`.

In [ ]:
data_path = '../data/processed/retornos_logaritmicos.parquet'

if not os.path.exists(data_path):
    raise FileNotFoundError(f"No se encontró el archivo: {data_path}")

log_returns = pd.read_parquet(data_path)
print("Dimensiones:", log_returns.shape)
log_returns.head()

## 3. Función TDA: normas L1 y L2 para H1

Función corregida y validada para Colab.

Calcula normas L1 y L2 del paisaje de persistencia generado a partir de ventanas deslizantes
sobre los retornos logarítmicos multivariados.

In [ ]:
def compute_lp_norms_sliding(
    returns_df: pd.DataFrame,
    window_size: int = 60,
    step: int = 1,
    maxdim: int = 1
):
    dates_center = []
    L1_vals = []
    L2_vals = []

    values = returns_df.values
    idx = returns_df.index
    n = len(returns_df)

    for start in tqdm(range(0, n - window_size + 1, step), desc=f"Ventanas w={window_size}"):
        end = start + window_size
        cloud = values[start:end, :]

        # Ripser
        res = ripser(cloud, maxdim=maxdim)
        dgms = res["dgms"]

        # H0 siempre existe
        dgm_H0 = dgms[0]
        dgm_H0 = dgm_H0[np.isfinite(dgm_H0[:, 1])]
        H0_list = [tuple(x) for x in dgm_H0]

        # H1 puede no existir
        if len(dgms) < 2:
            H1_list = []
        else:
            dgm_H1 = dgms[1]
            dgm_H1 = dgm_H1[np.isfinite(dgm_H1[:, 1])]
            H1_list = [tuple(x) for x in dgm_H1]

        # Estructura requerida por Persim
        dgms_persim = [H0_list, H1_list]

        # Paisaje de persistencia
        try:
            pl = PersLandscapeExact(dgms=dgms_persim, hom_deg=1)
            L1_vals.append(pl.p_norm(1))
            L2_vals.append(pl.p_norm(2))
        except:
            L1_vals.append(0.0)
            L2_vals.append(0.0)

        center_idx = start + window_size // 2
        dates_center.append(idx[center_idx])

    return pd.DataFrame(
        {"L1": L1_vals, "L2": L2_vals},
        index=pd.to_datetime(dates_center)
    )

print("Función TDA cargada correctamente.")

## 4. Ejecutar el TDA para varias ventanas deslizantes

Ventanas típicas utilizadas en la ponencia SEIO 2023.

In [ ]:
window_sizes = [30, 60, 90]
tda_results = {}

for w in window_sizes:
    print(f"\n=== Procesando ventana {w} días ===")
    tda_results[w] = compute_lp_norms_sliding(log_returns, window_size=w, step=1)

print("Cálculo completado.")

## 5. Guardar resultados en data/processed/

In [ ]:
output_path = '../data/processed/tda_landscapes_H1_norms.csv'

df_export = pd.concat({w: df for w, df in tda_results.items()}, axis=1)

df_export.to_csv(output_path)

print(f"Archivo guardado en: {output_path}")

## 6. Visualización rápida (opcional)

In [ ]:
plt.figure(figsize=(12,5))
for w, df_w in tda_results.items():
    plt.plot(df_w.index, df_w["L1"], label=f"w={w}")
plt.xlabel("Fecha")
plt.ylabel(r"$\\|L\\|_1$")
plt.title("Norma L1 de paisajes de persistencia (H1)")
plt.legend()
plt.grid(True)
plt.show()